In [39]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vogie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
news_dataset = pd.read_csv('news.csv')
news_dataset.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
news_dataset.shape

(6335, 4)

In [5]:
news_dataset.info

<bound method DataFrame.info of       Unnamed: 0                                              title  \
0           8476                       You Can Smell Hillary’s Fear   
1          10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2           3608        Kerry to go to Paris in gesture of sympathy   
3          10142  Bernie supporters on Twitter erupt in anger ag...   
4            875   The Battle of New York: Why This Primary Matters   
...          ...                                                ...   
6330        4490  State Department says it can't find emails fro...   
6331        8062  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
6332        8622  Anti-Trump Protesters Are Tools of the Oligarc...   
6333        4021  In Ethiopia, Obama seeks progress on peace, se...   
6334        4330  Jeb Bush Is Suddenly Attacking Trump. Here's W...   

                                                   text label  
0     Daniel Greenfield, a Shillman Journalism Fell

In [6]:
news_dataset = news_dataset.drop(['Unnamed: 0', 'title'], axis=1)

In [7]:
news_dataset.head()

,text,label
0,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,It's primary day in New York and front-runners...,REAL


In [8]:
port_stem = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [9]:
news_dataset['text'] = news_dataset['text'].apply(stemming)

In [10]:
print(news_dataset['text'])

0       daniel greenfield shillman journal fellow free...
1       googl pinterest digg linkedin reddit stumbleup...
2       u secretari state john f kerri said monday sto...
3       kayde king kaydeek novemb lesson tonight dem l...
4       primari day new york front runner hillari clin...
                              ...                        
6330    state depart told republican nation committe c...
6331    p pb stand plutocrat pentagon post oct wikimed...
6332    anti trump protest tool oligarchi reform alway...
6333    addi ababa ethiopia presid obama conven meet l...
6334    jeb bush suddenli attack trump matter jeb bush...
Name: text, Length: 6335, dtype: object


In [50]:
X = news_dataset['text']
Y = news_dataset['label']

In [48]:
print(X)

0       daniel greenfield shillman journal fellow free...
1       googl pinterest digg linkedin reddit stumbleup...
2       u secretari state john f kerri said monday sto...
3       kayde king kaydeek novemb lesson tonight dem l...
4       primari day new york front runner hillari clin...
                              ...                        
6330    state depart told republican nation committe c...
6331    p pb stand plutocrat pentagon post oct wikimed...
6332    anti trump protest tool oligarchi reform alway...
6333    addi ababa ethiopia presid obama conven meet l...
6334    jeb bush suddenli attack trump matter jeb bush...
Name: text, Length: 6335, dtype: object


In [13]:
print(Y)

0       FAKE
1       FAKE
2       REAL
3       FAKE
4       REAL
        ... 
6330    REAL
6331    FAKE
6332    FAKE
6333    REAL
6334    REAL
Name: label, Length: 6335, dtype: object


In [52]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

In [53]:
vectorizer = TfidfVectorizer()
tfid_X_train = vectorizer.fit_transform(X_train)
tfid_X_test = vectorizer.transform(X_test)

In [54]:
le = LabelEncoder();
Y_train = le.fit_transform(Y_train)
Y_test = le.transform(Y_test)

In [57]:
model = LogisticRegression()
model.fit(tfid_X_train, Y_train)

LogisticRegression()

In [59]:
X_train_prediction = model.predict(tfid_X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [60]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.952052091554854


In [61]:
X_test_prediction = model.predict(tfid_X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print(X_test_prediction)

[1 0 0 ... 1 1 1]


In [62]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9187056037884768


In [64]:
x_test_proba = model.predict_proba(tfid_X_test)
print(x_test_proba)

[[0.08161059 0.91838941]
 [0.75556537 0.24443463]
 [0.51905527 0.48094473]
 ...
 [0.07169051 0.92830949]
 [0.08846819 0.91153181]
 [0.03516876 0.96483124]]


In [66]:
X_new = tfid_X_test[3]

prediction = model.predict(X_new)
probabilities = model.predict_proba(X_new)
print(prediction)

if (prediction[0]==1):
  print('The news is Real')
else:
  print('The news is Fake')

#Pro Feature
print("Real%: " + str(probabilities[0][1]))
print("Fake%: " + str(probabilities[0][0]))

[0]
The news is Fake
Real%: 0.10796379751984128
Fake%: 0.8920362024801587


In [73]:
def fake_news_det(news):
    input_data = [news]
    vectorized_input_data = vectorizer.transform(input_data)
    prediction = model.predict(vectorized_input_data)
    prediction = le.inverse_transform(prediction)
    print(prediction)

In [74]:
fake_news_det("daniel greenfield shillman journal fellow free.")

['FAKE']


In [75]:
import pickle

In [26]:
news_dataset.to_csv("Cleaned_News_Dataset.csv")

In [76]:
pickle.dump(model, open('FakeNewsmodel.pkl', 'wb'))